<a href="https://colab.research.google.com/github/tproffen/ORCSGirlsPython/blob/master/Numbers/ChaosAndFractals-Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/tproffen/ORCSGirlsPython/blob/master/Images/Logo.png?raw=1" width="10%" align="right" hpsace="50">

# Numbers and Chaos

## Activity: The Logistical Map

Let us play with  numbers and unlock the secrets of the logistical map. Please make sure you run the cell with the setup commands below before running any other code in this notebook.


In [38]:
import ipywidgets as widgets
import plotly.graph_objects as go
import numpy as np

from google.colab import output

### Simple plotting

First we look at `Plotly` and how to graph data. In the past we have used `matplotlib`, but here we want to create interactive plots in Google Colab, so we use `Plotly` instead. Let us start with a simple line. The equation for a line is simply

$y=m*x + b$

where $m$ is the slope and $b$ is the y-axis intercept. Because we want to easily recalculate the points of the line for plotting, we create a function that takes the slope $m$ and intercept $b$, the range we want to cover in x (as xmin, xmax and dx) and returns a list of x and y values.

In [32]:
def line(m,b,xmin,xmax,dx):
    xvals = []                         # Empty list for x values
    yvals = []                         # Empty list for y values
    for x in np.arange(xmin,xmax,dx):  # Loop over points in x
        xvals.append(x)                # Add x vlaue to the list
        yvals.append(m*x+b)            # Calculate y and add to the list
    return xvals, yvals                # return both lists

Now we use that function to calculate the points and call it everytime the slider is moved. The code below will create sliders for $m$ and $b$ and allow the plot to be changed interactively :)

In [37]:
# Set ranges
xmin = -4.0
xmax = 4.0
dx = 0.05

(x,y) = line(1,0,xmin,xmax,dx)      # Initial line

# Making the graph
layout = go.Layout(title='Line Graph', yaxis_range=[min(y),max(y)], width=800, height=600)
data = go.Scatter(x=x, y=y, mode="lines", name='Line', xaxis='x', yaxis='y')
output.enable_custom_widget_manager()
graph = go.FigureWidget(data=[data], layout=layout)

# update routine
def update(change):
    (graph.data[0].x, graph.data[0].y) = line(mSlider.value, bSlider.value, xmin, xmax, dx)

# Making the sliders
mSlider = widgets.FloatSlider(value=1.0, min=-5, max=5.0, step=0.1, description='Slope m:', continuous_update=True, readout=True, readout_format='.1f')
bSlider = widgets.FloatSlider(value=1.0, min=-5, max=5.0, step=0.1, description='Intercept b:', continuous_update=True, readout=True, readout_format='.1f')
mSlider.observe(update, names='value')
bSlider.observe(update, names='value')
sliders = widgets.VBox([mSlider,bSlider])

display(widgets.HBox([graph,sliders]))

    'data': [{'mode': 'lines',
              'name': 'Line',
              'type…

# Now the fun - the Logistic Map

You have heard about the logisicatl map in the class or you can watch <a href="https://youtu.be/ovJcsL7vyrk">this awesome video</a>.

Here is the equation:
$x_{n+1} = r x_{n}(1 - x_{n})$

Following the notation used in class, $r$ stands for the growth rate and $x$ is population. This is not the actual number, but the fraction of the maximum possible population, so it ranges from 0 to 1.

#### Step 1: Visualize the function

Let's define a function `growth` and plot how the population next year depends on the population this year;

In [ ]:
def growth(x,r):
    return r*x*(1-x)

In [ ]:
r = 2.0   # Growth rate of 2
x = 0.4   # Initial population 40% (or 0.4) of the possible maximum

xpts = np.arange(0.0,1.1,0.1)           # Values for x from 0 to 1 (note the 1.1)
ypts = [growth(x,r) for x in xpts]      # One line loop to calculate all the y's

fig, ax = plt.subplots(figsize=(6,4))   # Making the new plot
plt.xlabel('x_n')
plt.ylabel('x_n+1')

ax.plot(xpts,ypts)

Woohoo, just like in the video. He have a 'single hump function' with a negative feedback, meaning when the population gets too big the dowturn of the parabula will reduce the population as you can see on the graph.

#### Step 2: Behaviour as time goes on

We are most interested in the long term behaviour of the function as we repeatley call it. In other words what happens to the population after a long time. In the function below we calculate the value of $x_{n} = r x_{n-1}(1 - x_{n-1})$ for all $n$. Note that we have rewritten this so the value we calculate os $n$ and it depends on the earlier value $n-1$ which is how we implement it by using the list element `[-1]`.

In [ ]:
def growthtime(x0,r,nmax):
    vals=[x0]                                  # First element is the initial pop.
    for i in range(nmax-1):                    # Loop to the maximum number nmax we want.
                                               #   the -1 is because we already added x0
        vals.append(r*vals[-1]*(1-vals[-1]))   # Append the calculated vale
    return vals                                # Return all values

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
plt.xlabel('Time')
plt.ylabel('Population')

@widgets.interact(x0=(0.1,0.9,0.1),r=(0.5,4.0,0.01))
def update(x0=0.4, r=1.2):
    [l.remove() for l in ax.lines]
    ax.plot(growthtime(x0,r,100), color="blue", marker='o')    # Adding some markers

Explore how the function depends on `x0` and `r`. Anything spooky 😨

#### Step 3: Exploring as function of growth rate $r$

You might have seen strange behaviour for certain values of $r$, so next we explore the resulting populations after some time. Because there are multiple values the function oscillates around, we simply loop ofer $r$ and then plot the last `nplot` points ion a scatter plot.

In [ ]:
x0 = 0.4         # Starting population
nmax  = 300      # Maximum value of n for calculate
nplot = 150      # First point to plot, so we plot nplot:nmax

rvals=[]         # List for values of r
lvals=[]         # List for resulting values of x (there will be nmax values for each r)

for r in np.arange(0.9,4.0,0.001):              # Loop over r - note the fine grid
    lvals.append(growthtime(x0,r,nmax)[nplot:]) # Getting list of x starting at nplot
    rvals.append([r]*(nmax-nplot))              # We need the same number of r's so
                                                # lists are the same length

# Now we need to turn the list of lists into one contineous list for plotting
lvals = np.array(lvals).ravel()
rvals = np.array(rvals).ravel()

All that is left is to plot. Adjust marker size and color below. Also explore changing nmax, nplot and the step size in r. Enjoy.

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
plt.scatter(rvals,lvals,s=0.02,color='purple')

### How cool was that ..